In [1]:
#TDL
"""
Whats missing?
-get the big grid search running
-fix the non start issue
-
"""

'\nWhats missing?\n-get the big grid search running\n-fix the non start issue\n-\n'

In [2]:
from concurrent.futures import ThreadPoolExecutor

def evaluate_net(net, x, y):
    predictions = net.predict(x, config["inference_steps"])[0]
    correct_count = len(x) - (np.sum(np.abs(y - predictions)) / 2)
    performance = correct_count / len(x)
    return (performance, net)

def evaluate_performance_fast(population, x, y):
    with ThreadPoolExecutor() as executor:
        performances = list(executor.map(evaluate_net, population, [x] * len(population), [y] * len(population)))

    print("Evaluation done!")

    # Sort by best performance
    performances.sort(key=lambda x: x[0], reverse=True)
    return performances


# Pseudo-code for the new version of the evaluate_performance function




In [3]:
import numpy as np
import tensorflow as tf
import tensorflow.keras
import matplotlib.pyplot as plt
import sys
import os
from non_layered_neural_net import nlnn
import tracemalloc
import functools
from typing import Callable


In [4]:
def memory_usage_decorator(func: Callable):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        tracemalloc.start()
        before_memory = get_memory_usage()

        result = func(*args, **kwargs)

        after_memory = get_memory_usage()
        tracemalloc.stop()

        print(f"Function '{func.__name__}' memory usage:")
        print(f"Before: {before_memory} MB")
        print(f"After: {after_memory} MB")
        print(f"Memory increase: {after_memory - before_memory} B")

        return result

    return wrapper

In [5]:
#remove later
sys.argv[0] = "1"

In [6]:
#Run_hyperparameters
training_run = int(sys.argv[0])
neuron_count=300
connection_probability_dropoff=3
connection_probability_scalar=0.00003
input_layer_connectivity_multiplyer=50
output_layer_connectivity_multiplyer=20
weight_initialisation_range=2
distances_from_input_output_layer_to_main_neuron_field=0.1
hidden_neuron_connections = 7
input_neuron_connections = 10
output_neuron_connections = 10
inference_steps = 8

n_closes_neurons_connection_probability="connection_prob"  #"connection_prob" /"n_closest"
activation_function="sigmoid" #relu

generation_size=10
n_survivors=3
mutation_range=0.1
training_set_size=1000
mutation_range_reducing_interval="none"
mutation_range_reducing_factor="none"

reducing_mutaiton_range="no"
stochastic_mutation_range="yes"
multiple_training_sets="yes"
keep_best_of_n_generations_keep_n_best="keep_n_best"
allow_topological_modification="no"
non_uniform_distribution_in_stochastic_mutation_range="no"


In [7]:
#this function produces a configuration of parameters based on an input number. this allows the python script to be run with just one parameter instead of 8

def get_configuration(index):
    base_config = {
        "neuron_count": 1000,
        "connection_probability_dropoff": 3.0,
        "hidden_neuron_connections": 6,
        "inference_steps": 8,
        "n_survivors": 3,
        "activation_function": "leaky_relu",
    }

    params_to_evaluate = list(base_config.keys())
    param_index = index // 5
    run_index = index % 5

    param = params_to_evaluate[param_index]

    if param == "neuron_count":
        base_config[param] = int((np.linspace(9, 19, 5) ** 2.5)[run_index])
    elif param == "connection_probability_dropoff":
        base_config[param] = np.linspace(1, 4, 5)[run_index]
    elif param == "hidden_neuron_connections":
        base_config[param] = int(np.linspace(20, 40, 5)[run_index])
        base_config["n_closes_neurons_connection_probability"] = "n_closest"
    elif param == "inference_steps":
        base_config[param] = int(np.linspace(6, 20, 5)[run_index])
    elif param == "n_survivors":
        base_config[param] = int(np.linspace(1, 12, 5)[run_index])
    elif param == "activation_function":
        options = ["relu", "leaky_relu"]
        base_config[param] = options[run_index % len(options)]

    return base_config


In [8]:
config = {         "training_run":training_run,
                   "neuron_count":neuron_count,
                   "connection_probability_dropoff":connection_probability_dropoff,
                   "connection_probability_scalar":connection_probability_scalar,
                   "input_layer_connectivity_multiplyer":input_layer_connectivity_multiplyer,
                   "output_layer_connectivity_multiplyer":output_layer_connectivity_multiplyer,
                   "weight_initialisation_range":weight_initialisation_range,
                   "n_closes_neurons_connection_probability":n_closes_neurons_connection_probability,
                   "hidden_neuron_connections" : hidden_neuron_connections,
                   "input_neuron_connections" : input_neuron_connections,
                   "output_neuron_connections" : output_neuron_connections,
                   "inference_steps" : inference_steps,
                   "activation_function":activation_function,
                   "generation_size":generation_size,
                   "n_survivors":n_survivors,
                   "mutation_range":mutation_range,
                   "training_set_size":training_set_size,
                   "mutation_range_reducing_interval":mutation_range_reducing_interval,
                   "mutation_range_reducing_factor":mutation_range_reducing_factor,
                   "reducing_mutaiton_range":reducing_mutaiton_range,
                   "stochastic_mutation_range":stochastic_mutation_range,
                   "multiple_training_sets":multiple_training_sets,
                   "allow_topological_modification":allow_topological_modification
            }



In [9]:
settings = get_configuration(int(sys.argv[0]))
for i in settings.keys():
    config[i]=settings[i]
for i in config.keys():   
    print(i,":",config[i])

training_run : 1
neuron_count : 448
connection_probability_dropoff : 3.0
connection_probability_scalar : 3e-05
input_layer_connectivity_multiplyer : 50
output_layer_connectivity_multiplyer : 20
weight_initialisation_range : 2
n_closes_neurons_connection_probability : connection_prob
hidden_neuron_connections : 6
input_neuron_connections : 10
output_neuron_connections : 10
inference_steps : 8
activation_function : leaky_relu
generation_size : 10
n_survivors : 3
mutation_range : 0.1
training_set_size : 1000
mutation_range_reducing_interval : none
mutation_range_reducing_factor : none
reducing_mutaiton_range : no
stochastic_mutation_range : yes
multiple_training_sets : yes
allow_topological_modification : no


In [10]:
#load dataset
def load_local_mnist_data(path):
    with np.load(path) as f:
        x_train, y_train = f['x_train'], f['y_train']
        x_test, y_test = f['x_test'], f['y_test']
    return (x_train, y_train), (x_test, y_test)
(train_X, train_y), (test_X, test_y) = load_local_mnist_data('mnist.npz')
#one hot encode
def one_hot_encode(x):
    out = np.zeros((len(x), max(x)+1))
    for i in range(len(x)):
        out[i][x[i]] = 1
    return out

y_train_ohe = one_hot_encode(train_y)
y_test_ohe = one_hot_encode(test_y)
#flatten images
x_train = train_X.reshape(len(train_X), 28*28)
x_test = test_X.reshape(len(test_X), 28*28)

x_test = np.array_split(x_test, 10)
y_test_ohe = np.array_split(y_test_ohe, 10)

x_val = x_train[:10000]
y_val = y_train_ohe[:10000]

In [11]:
def create_population(population_size):
    population = []
    print("Creating networks")
    for i in range(population_size):
        net = nlnn(output_neurons = 10, hidden_neurons = config["neuron_count"])
        if config["n_closes_neurons_connection_probability"] == "connection_prob":
            net.initialise_structure(connection_probability_dropoff=config["connection_probability_dropoff"], connection_probabily_scalar=config["connection_probability_scalar"], input_connection_prob_multiplyer = config["input_layer_connectivity_multiplyer"], output_connection_prob_multiplyer = config["output_layer_connectivity_multiplyer"])
        elif config["n_closes_neurons_connection_probability"] == "n_closest":
            net.initialise_structure_n_closest(hidden_neuron_connections = config["hidden_neuron_connections"], input_neuron_connections = config["input_neuron_connections"], output_neuron_connections = config["output_neuron_connections"])
        #net.initialise_randomly()
        population.append(net)
        print("|", end="")

    print("done!")
    return population


In [19]:
def get_perf(t):
        return t[0]

#measure performance of all the networks
def evaluate_performance(population, x, y):
    performances = []
    print("evaluating performances", end="")
    for net in population:
        predictions = net.predict(x, config["inference_steps"])[0]
        correct_count = len(x)-(np.sum(np.abs(y - predictions))/2)
        performances.append((correct_count/len(x), net))
        print("|", end="")
    print(" done!", end=" ")
    #sort by best performance
    performances.sort(key = get_perf, reverse = True)
    return performances

# Revised pseudo-code for the new version of the evaluate_performance function
def evaluate_performance_parallel(population, x, y, config):
    # Initialize neuron_values for all networks in the population
    dim_matrix = population[0].dim_matrix  # Assuming all networks have the same dim_matrix
    neuron_values = np.zeros((len(population), len(x), dim_matrix))
    neuron_values[:, :, :x.shape[1]] = np.repeat(x[np.newaxis, :, :], len(population), axis=0)
    
    # Initialize performances list
    performances = []
    print("Evaluating performances in parallel", end="")
    
    # Propagation Steps
    for step in range(config["inference_steps"]):
        # Batched Matrix Multiplication and Non-linearity (Leaky ReLU in this case)
        z = np.matmul(neuron_values, np.array([net.adj_matrix for net in population]))  # Batched multiplication
        z = np.where(z > 0, z, z * 0.01)  # Leaky ReLU
        neuron_values[:, :, :x.shape[1]] = np.repeat(x[np.newaxis, :, :], len(population), axis=0)  # Update with x
    
    # Scoring
    for i, net in enumerate(population):
        individual_output = neuron_values[i]  # Replace with your method of extracting output
        predictions = np.argmax(individual_output, axis=1)  # Your prediction logic here
        predictions = np.eye(net.output_neurons)[predictions]
        correct_count = len(x) - (np.sum(np.abs(y - predictions)) / 2)
        performances.append((correct_count / len(x), net))
        print("|", end="")
    
    print(" done!", end=" ")
    
    # Sort by best performance
    performances.sort(key=lambda x: x[0], reverse=True)
    
    return performances

# Example usage (replace with your actual data and network objects)
# evaluate_performance_parallel(population, x_test[test_set], y


In [13]:
def repopulate(evaluated_networks, mutation_range, n):
    offspring_per_network = int(population_size/n)
    parents = [i[1] for i in evaluated_networks[:n]]
    offspring = []
    for net in parents:
        net_offspring = net.reproduce(min(offspring_per_network, population_size - len(offspring)), mutation_range)
        offspring.extend(net_offspring)
    next_gen = parents + offspring
    next_gen = next_gen[:population_size]
    return next_gen


In [14]:
performance_over_time = []
test_sets_used = []
mutation_ranges = []
validation_accuracy = []

In [16]:
neuron_count = 200
inference_steps = 3
#Training loop
    #loading hyperparameters
n = config["n_survivors"]
mutation_range = config["mutation_range"]
population_size = config["generation_size"]
print_graphs = False
population_size = 10

networks = create_population(population_size)
networks = evaluate_performance(networks, x_test[0], y_test_ohe[0])
print("best performer of this generation :", networks[0][0])
performance_over_time.append(np.array(networks)[:,0])
networks = repopulate(networks, mutation_range,n)


Creating networks
||||||||||done!
evaluating performances|||||||||| done! best performer of this generation : 0.133


In [20]:

generations = 1000
test_set = 0
for gen in range(generations):
    if(config["reducing_mutaiton_range"]=="yes"):
        if(gen%config["mutation_range_reducing_interval"]==0 and gen!=0): 
            mutation_range/=config["mutation_range_reducing_factor"]
            print("decreasing mutation range from",mutation_range*config["mutation_range_reducing_factor"],"to",mutation_range)
    print("generation "+str(len(performance_over_time)+1), end=" ")

    if config["multiple_training_sets"] == "yes"  and gen!=0:
        test_set = np.random.randint(10)
    test_sets_used.append(test_set)
    print(" test set:",test_set,end=" ")
    networks = evaluate_performance_parallel(networks, x_test[test_set], y_test_ohe[test_set], config)
    
    print(" best:", networks[0][0] )#, "second:", evaluated_networks[1][0], "third:", evaluated_networks[2][0])
    performance_over_time.append(np.array(networks)[:,0])
    generational_mutation_range = mutation_range
    if config["stochastic_mutation_range"]=="yes": #change back
        generational_mutation_range = np.random.rand() * mutation_range
    mutation_ranges.append(generational_mutation_range)
    print("mutating in range:", generational_mutation_range)
    networks = repopulate(networks, generational_mutation_range, config["n_survivors"])
    if(gen%10 == 0):
        val_accuracy = evaluate_performance([networks[0]], x_val, y_val)[0][0]
        validation_accuracy.append(val_accuracy)
    if(gen%10==0):
        plt.plot(np.array(performance_over_time), alpha= 0.1)
        plt.plot(np.array(performance_over_time)[:,0])
        plt.show()
        print("average best of last 100 generations",np.average(np.array(performance_over_time)[-100:,0]))  
        plt.plot(validation_accuracy)
        plt.show()


generation 2  test set: 0 Evaluating performances in parallel

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 10 is different from 1242)

In [ ]:
#visualising performance across the 10 different sets
def visualise_performance():
    performance_hist = np.array(performance_over_time)[-len(test_sets_used):,0]
    set_perf = []
    for i in range(10):
        set_perf.append([])
    for i in range(len(test_sets_used)-10):
        set_perf[test_sets_used[i]].append(performance_hist[i])

    for i in set_perf:
        plt.plot(i, alpha=0.8)
    plt.show()
    """
    plt.violinplot(set_perf)
    plt.show()
    """
    performance_hist = np.array(performance_over_time)[-len(test_sets_used):]
    performance_changes = []
    for i in range(len(performance_hist)-1):
        performance_changes.append(-(np.average(performance_hist[i])-np.average(performance_hist[i+1])))

    plt.scatter(np.array(performance_changes)[:len(mutation_ranges)], np.array(mutation_ranges)[:len(performance_changes)])
    plt.xlabel('change in perform')
    plt.ylabel('Y-axis label')
    plt.title("avg performance increase vs. mutation ranges")
    plt.show()

    plt.violinplot(np.array(mutation_ranges))
    plt.title("frequency of mutation ranges")
    plt.show()


    plt.plot(performance_over_time, alpha= 0.1)
    plt.plot(np.array(performance_over_time)[:,0])
    plt.title("performance of ranked individuals")
    plt.show()


    plt.plot(np.average(np.split(np.array(performance_over_time)[:6400,0], 40),axis =1))
    plt.title("performannce over time, smoothed")
    plt.show()
    
visualise_performance()

In [ ]:
config["duration"] = duration

folder_name = 'run_'+str(training_run)
suffix = 1

while os.path.exists(folder_name):
    suffix += 1
    folder_name = f"{folder_name}_{suffix}"

os.makedirs(folder_name)


performance_over_time_array = np.array(performance_over_time)

np.savetxt(folder_name+'/training_run_'+str(training_run)+'_performance.csv', performance_over_time_array, delimiter=',')
np.savetxt(folder_name+'/training_run_'+str(training_run)+'_test_sets_used.csv', test_sets_used, delimiter=',')
np.savetxt(folder_name+'/training_run_'+str(training_run)+'_mutation_ranges.csv', mutation_ranges, delimiter=',')
np.savetxt(folder_name+'/training_run_'+str(training_run)+'_best_weights.csv', evaluated_networks[0][1].adj_matrix.toarray(), delimiter=',')
np.save(folder_name+'/training_run_'+str(training_run)+'_config.npy', np.array(config))